# Convert Summary Statistics to Json for Interactive Locus Zoom Tool
- **Author(s)** - Frank Grenn
- **Date Started** - December 2020
- **Quick Description:** subsets data from a summary statistics file and converts to json for interactive locus zoom. 

## (1) Setup

In [4]:
library(rjson)
library(data.table)

In [15]:
#names of columns in summary stats file
chr_name <- "CHR"
position_name <- "BP"
pvalue_name <- "LOG_P"
ref_name <- "REF"
variant_name <- "RSID"

#is the p value column already -log10?
p_is_log <- TRUE


In [16]:
#output file prefix
out_prefix <-"/path/to/ms_sumstats"


In [17]:
#summary stats file
data <- as.data.frame(fread("/path/to/discovery_metav3.0.meta"))

In [18]:
#how many bps to go up and downstream of each region
bprange <- 1000000


#regions to take from the summary stats
regions <- data.frame(chr=c(1,3,3,5,12,14,17,17),bp=c(154983036,18798848,121765368,133891282,123604053,88523488,40529835,43407670))
head(regions)

,chr,bp
,<dbl>,<dbl>
1,1,154983036
2,3,18798848
3,3,121765368
4,5,133891282
5,12,123604053
6,14,88523488


## (2) Format Summary Statistics

In [19]:
#remove any rows with a NA
complete <- data[complete.cases(data),]

#log calculations if needed
if(!p_is_log)
{
  complete$"log_pvalue" <- -1 * log10(complete[,names(complete)==pvalue_name])
} else {
  names(complete)[names(complete) == pvalue_name] <- "log_pvalue"
}

#rename columns
names(complete)[names(complete) == chr_name] <- "chromosome"
names(complete)[names(complete) == position_name] <- "position"
names(complete)[names(complete) == ref_name] <- "ref_allele"
names(complete)[names(complete) == variant_name] <- "variant"

#chromosomes to strings
complete$chromosome <- as.character(complete$chromosome)

#final df to use for json
df <- complete[order(complete$chromosome,complete$position),c("chromosome","log_pvalue","position","ref_allele","variant")]
print(dim(data))
print(dim(df))

[1] 7818616      13
[1] 7805651       5


## (3) Get region sum stats and save as json for locuszoom
####   (a) all regions to one file
####   or
####   (b) all regions to separate files

## (a) All Regions to One File

In [20]:
sub_df <- data.frame()


In [21]:
for (r in 1:nrow(regions))
{
    row <- regions[r,]
    print(row)
    region_df <- df[(df$chromosome==as.character(row$chr) & df$position < row$bp +bprange & df$position > row$bp -bprange),]
    print(dim(region_df))
    sub_df <- rbind(sub_df,region_df)

    print("new:")
    print(dim(sub_df))

}

  chr        bp
1   1 154983036
[1] 3320    5
[1] "new:"
[1] 3320    5
  chr       bp
2   3 18798848
[1] 4335    5
[1] "new:"
[1] 7655    5
  chr        bp
3   3 121765368
[1] 5500    5
[1] "new:"
[1] 13155     5
  chr        bp
4   5 133891282
[1] 4696    5
[1] "new:"
[1] 17851     5
  chr        bp
5  12 123604053
[1] 4632    5
[1] "new:"
[1] 22483     5
  chr       bp
6  14 88523488
[1] 5832    5
[1] "new:"
[1] 28315     5
  chr       bp
7  17 40529835
[1] 3984    5
[1] "new:"
[1] 32299     5
  chr       bp
8  17 43407670
[1] 3000    5
[1] "new:"
[1] 35299     5


In [22]:
convert_list <- list(data = sub_df)


json <- toJSON(convert_list)
writeLines(json, file(paste0(out_prefix,".json")))

## (b) All Regions to Separate Files

In [23]:
for (r in 1:nrow(regions))
{
    row <- regions[r,]
    print(row)
    region_df <- df[(df$chromosome==as.character(row$chr) & df$position < row$bp +bprange & df$position > row$bp -bprange),]
    print(dim(region_df))

    
    convert_list <- list(data = region_df)


    json <- toJSON(convert_list)
    new_out_prefix <- paste0(out_prefix,"_chr",row$chr,"_",row$bp -bprange,"_",row$bp+bprange)
    
    writeLines(json, file(paste0(new_out_prefix,".json")))

}

  chr        bp
1   1 154983036
[1] 3320    5
  chr       bp
2   3 18798848
[1] 4335    5
  chr        bp
3   3 121765368
[1] 5500    5
  chr        bp
4   5 133891282
[1] 4696    5
  chr        bp
5  12 123604053
[1] 4632    5
  chr       bp
6  14 88523488
[1] 5832    5
  chr       bp
7  17 40529835
[1] 3984    5
  chr       bp
8  17 43407670
[1] 3000    5
